I'm trying to figure out what the deal is with loss modules in TorchRL.

Docs on loss modules: <https://pytorch.org/rl/stable/reference/objectives.html>

Here's what I've gathered so far:

### Trainable Functions

In RL, trainable functions are used to estimate things. The actual thing that the trainable function is trying to estimate is called the target. Two examples of trainable functions are a policy function and a value function.

#### Policy function

Estimating an optimal policy function is basically the whole purpose of RL. The policy function is what the agent uses to decide which action to take at each step, given the current observation. The policy function has trainable parameters inside of it to calculate an action for the agent to take. For instance, often the policy function is a neural network with trainable weights and biases. During the training loop we iteratively update these paramaters according to some optimization algorithm, in order to make the policy function more closely approximate whatever behavior we are trying to promote. The target of the policy function is the theoretical optimal strategy.

#### Value function

Often the training loop will use a value function (or value estimator) to estimate the expected value of a particular action at a particular step, assuming that the agent will continue to follow the given policy for the remaining steps. The value function has to somehow encapsulate information about the value of the entire tree of possible future trajectories to properly estimate the value of an action at the current step. The value function is used to train the policy function to promote good sequences of actions, not just good individual actions. The more accurate the value function is, the more accurately it can inform changes to the policy parameters. The value function itself has trainable parameters (also may be a neural network) so that the value estimation can be improved during the training loop. The target of the value function is the "actual" theoretical value of an action given an observation.

### Loss Functions

A loss function is a function which determines how wrong a trainable function is. It gives us the difference between the target and the estimation given by the trainable function--this difference is called the loss. During the training loop we can use the loss to back propagate and update the parameters of the trainable function. Over time, the loss is supposed to decrease, meaning that the output of the trainable function more closely matches the target.

#### DQN Loss

Deep Q-learning (or DQN, where N stands for "network") is a particular algorithm for RL. In this case, Q represents the value of an action in a particular state, and the value function is trying to estimate Q. This algorithm uses a deep conv net to implement the value function.

DQN has a loss function for training the value function. The loss function is commonly implemented as either L1, smooth-L1, or L2, but nevermind what these mean--they are just straightforward formulae for calculating some kind of difference between the target and the estimate.

### TorchRL LossModule

TorchRL has a class called [`LossModule`](https://pytorch.org/rl/stable/reference/generated/torchrl.objectives.LossModule.html), which is a generic parent class, whose derived classes implement particular loss functions. There are a lot of different derived classes that each have a specialized purpose for calculating the losses of various different kinds of trainable functions.

One of the derived classes is [`DQNLoss`](https://pytorch.org/rl/stable/reference/generated/torchrl.objectives.DQNLoss.html), which implements the loss function for the DQN algorithm's value function.

`DQNLoss` holds onto a reference to the value function. That way, to calculate the loss for that value function, we just have to pass target data (actual rewards from the environment, which the value function is trying to estimate) into the module, rather than having to provide the value function every time we call it.

Upon initialization, `DQNLoss` allows you to choose whether the loss function is L1, smooth-L1, or L2.

In general, a `LossModule` may calculate a loss that incorporates more than one trainable function. For instance, [`DDPGLoss`](https://pytorch.org/rl/stable/reference/generated/torchrl.objectives.DDPGLoss.html) calculates a loss from both a policy function and a value function.